In [12]:
# Tensorflow: 네트워크 모델 만들기
import cv2

import numpy as np

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import graph_util
from tensorflow.python.platform import gfile
from tensorflow.keras.utils import to_categorical

from glob import glob

tf.logging.set_verbosity(tf.logging.ERROR)

# set hyper parameters
learning_rate = 1e-3
training_epochs = 20
batch_size = 100

In [37]:
# Load data
def load_data(filepath):
    images = []
    labels = []
    
    label = 0
    
    for path in glob(filepath + "*"):
        for file in glob(path + "/*.jpg"):
            image = cv2.imread(file)
            images.append(image)
            
            labels.append(label)
        label += 1
    
    return np.array(images), np.array(labels).reshape(-1, 1)

train_data, train_label = load_data("data/train/")
print(train_data.shape, train_label.shape)
train_label = to_categorical(train_label)
print(train_data.shape, train_label.shape)

(50000, 32, 32, 3) (50000, 1)
(50000, 32, 32, 3) (50000, 10)


In [38]:
# Model configuration
X = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 3], name="data")
Y = tf.placeholder(tf.float32, [None, 10])

conv1 = tf.layers.conv2d(inputs=X, filters=10, kernel_size=[3, 3], padding="same", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, padding="same")

conv2 = tf.layers.conv2d(pool1, 20, [3, 3], padding="same", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(conv2, [2, 2], 2, "same")

fc1 = tf.contrib.layers.flatten(pool2)
fc2 = tf.layers.dense(inputs=fc1, units=200, activation=tf.nn.relu)
logits = tf.layers.dense(fc2, 10, tf.nn.relu)
output = tf.nn.softmax(logits, name="probability")

# 비용 함수(cost func) or 손실 함수(loss func) 지정
cost = tf.reduce_mean(input_tensor=tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

# 최적화기 지정
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss=cost)

In [39]:
# Training
sess = tf.Session()
sess.run(fetches=tf.global_variables_initializer())
total_batch = int(len(train_data) / batch_size)  # 50000 / 100 = '500 -> total_batch'

print("Start learning!")
for epoch in range(training_epochs):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs = train_data[batch_size * i:batch_size * (i+1)]
        batch_ys = train_label[batch_size * i:batch_size * (i+1)]
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val
    
    print("Epoch: {}, Avg. Cost = {:.4f}".format(epoch + 1, total_cost/total_batch))

print("Learning finished!")

Start learning!
Epoch: 1, Avg. Cost = 14.8842
Epoch: 2, Avg. Cost = 2.3018
Epoch: 3, Avg. Cost = 6.3687
Epoch: 4, Avg. Cost = 2.2998
Epoch: 5, Avg. Cost = 2.9997
Epoch: 6, Avg. Cost = 14.8624
Epoch: 7, Avg. Cost = 2.3026
Epoch: 8, Avg. Cost = 2.3026
Epoch: 9, Avg. Cost = 2.3026
Epoch: 10, Avg. Cost = 2.3026
Epoch: 11, Avg. Cost = 2.3026
Epoch: 12, Avg. Cost = 2.3026
Epoch: 13, Avg. Cost = 2.3026
Epoch: 14, Avg. Cost = 2.3026
Epoch: 15, Avg. Cost = 2.3026
Epoch: 16, Avg. Cost = 2.3026
Epoch: 17, Avg. Cost = 2.3026
Epoch: 18, Avg. Cost = 2.3026
Epoch: 19, Avg. Cost = 2.3026
Epoch: 20, Avg. Cost = 2.3026
Learning finished!


In [ ]:
# Freeze variables and save pb file
output_graph_def = graph_util.convert_variables_to_constants(
    sess=sess, input_graph_def=sess.graph_def, output_node_names=["probability"])
with gfile.FastGFile("./models/dacon_practice.pb", "wb") as f:
    f.write(output_graph_def.SerializeToString())

print("dacon_practice.pb file is created successfully!")

In [ ]:
# Case #2에서 만든 "mnist_cnn.pb" 모델 불러오기
net = cv2.dnn.readNet("models/dacon_practice.pb")

if net.empty():
    print("Network load failed!")

In [ ]:
image = np.zeros((400, 400), np.uint8)

cv2.imshow("image", image)
cv2.setMouseCallback("image", on_mouse)

while True:
    key = cv2.waitKey()
    
    if key == 27:
        break
    elif key == ord(" "):
        # MNIST 모델은 입력 데이터를 0~1 사이로 정규화하여 입력을 받기 때문에 여기서도 입력 픽셀 값을 0~1 사이로 정규화 한다.
        blob = cv2.dnn.blobFromImage(norm_digit(image), 1/255., (28, 28))
        net.setInput(blob)
        probability = net.forward()
        
        # return: min value, max value, min location, max location
        _, max_value, _, max_loc = cv2.minMaxLoc(probability)
        digit = max_loc[0]
        
        print(f"{digit} ({max_value * 100:4.2f})")
        
        image.fill(0)
        cv2.imshow("image", image)

cv2.destroyAllWindows()
cv2.waitKey(1)